# Compustat World Firms Industry Classification

This file was created to make parallesim easier.

##  Import Libraries and Load Datasets

In [1]:
# Data analysis
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter
from scipy.stats import uniform, loguniform
from skopt.space import Real, Categorical, Integer
import pickle


# Preprocessing & Splitting
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler, MinMaxScaler
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split, GroupShuffleSplit, StratifiedGroupKFold
from sklearn.decomposition import PCA

# Modeling
from skopt import BayesSearchCV 
import xgboost
from sklearn.linear_model import LogisticRegression  
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score,f1_score, precision_score, recall_score, classification_report, confusion_matrix

# Utils
from utils import *


In [2]:
compustat = pd.read_pickle('../data/compustat_ftreng.pkl')

In [3]:
compustat.shape

(2748178, 139)

## Model Fitting

### Preprocessing

In [4]:
year_ftrs = ['fyearq','fqtr']
num_ftrs = ['saleq', 'gpq', 'oiadpq', 'oibdpq', 'cogsq',
       'xoprq', 'atq', 'seqq', 'dlcq', 'dlttq', 'capxy', 'oancfy', 'gpm',
       'opm', 'ocfm', 'roa', 'roe', 'cd_ratio', 'ca_ratio', 'fca_ratio',
       'fce_ratio', 'fcd_ratio', 'fcs_ratio', 'tat', 'cr', 'tdr', 'der',
       'gpm_lag1', 'gpm_lag2', 'gpm_lag3', 'gpm_lag4', 'opm_lag1',
       'opm_lag2', 'opm_lag3', 'opm_lag4', 'ocfm_lag1', 'ocfm_lag2',
       'ocfm_lag3', 'ocfm_lag4', 'roa_lag1', 'roa_lag2', 'roa_lag3',
       'roa_lag4', 'roe_lag1', 'roe_lag2', 'roe_lag3', 'roe_lag4',
       'fca_ratio_lag1', 'fca_ratio_lag2', 'fca_ratio_lag3',
       'fca_ratio_lag4', 'fce_ratio_lag1', 'fce_ratio_lag2',
       'fce_ratio_lag3', 'fce_ratio_lag4', 'fcd_ratio_lag1',
       'fcd_ratio_lag2', 'fcd_ratio_lag3', 'fcd_ratio_lag4',
       'fcs_ratio_lag1', 'fcs_ratio_lag2', 'fcs_ratio_lag3',
       'fcs_ratio_lag4', 'tat_lag1', 'tat_lag2', 'tat_lag3', 'tat_lag4',
       'cr_lag1', 'cr_lag2', 'cr_lag3', 'cr_lag4', 'tdr_lag1', 'tdr_lag2',
       'tdr_lag3', 'tdr_lag4', 'der_lag1', 'der_lag2', 'der_lag3',
       'der_lag4', 'gpm_mean_4Q', 'gpm_std_4Q', 'gpm_mean_8Q',
       'gpm_std_8Q', 'opm_mean_4Q', 'opm_std_4Q', 'opm_mean_8Q',
       'opm_std_8Q', 'ocfm_mean_4Q', 'ocfm_std_4Q', 'ocfm_mean_8Q',
       'ocfm_std_8Q', 'roa_mean_4Q', 'roa_std_4Q', 'roa_mean_8Q',
       'roa_std_8Q', 'roe_mean_4Q', 'roe_std_4Q', 'roe_mean_8Q',
       'roe_std_8Q', 'fca_ratio_mean_4Q', 'fca_ratio_std_4Q',
       'fca_ratio_mean_8Q', 'fca_ratio_std_8Q', 'fce_ratio_mean_4Q',
       'fce_ratio_std_4Q', 'fce_ratio_mean_8Q', 'fce_ratio_std_8Q',
       'fcd_ratio_mean_4Q', 'fcd_ratio_std_4Q', 'fcd_ratio_mean_8Q',
       'fcd_ratio_std_8Q', 'fcs_ratio_mean_4Q', 'fcs_ratio_std_4Q',
       'fcs_ratio_mean_8Q', 'fcs_ratio_std_8Q', 'tat_mean_4Q',
       'tat_std_4Q', 'tat_mean_8Q', 'tat_std_8Q', 'cr_mean_4Q',
       'cr_std_4Q', 'cr_mean_8Q', 'cr_std_8Q', 'tdr_mean_4Q',
       'tdr_std_4Q', 'tdr_mean_8Q', 'tdr_std_8Q', 'der_mean_4Q',
       'der_std_4Q', 'der_mean_8Q', 'der_std_8Q']

#cat_ftrs = ['loc','curcdq']  # maybe not include country for now
cat_ftrs = ['curcdq'] 


In [5]:
minmax_transformer = Pipeline(steps=[
    ('scaler', MinMaxScaler())])
numeric_transformer = Pipeline(steps=[
    ('scaler', StandardScaler())])
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('onehot', OneHotEncoder(sparse_output=True, handle_unknown='ignore'))])
preprocessor1 = ColumnTransformer(
    transformers=[
        ('minmax', minmax_transformer, year_ftrs),
        ('num', numeric_transformer, num_ftrs),
        ('cat', categorical_transformer, cat_ftrs)])

numeric_transformer2 = Pipeline(steps=[
    ('scaler', StandardScaler()),
    ('imputer', SimpleImputer(strategy='constant', fill_value=0))])
categorical_transformer2 = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('onehot', OneHotEncoder(sparse_output=False, handle_unknown='ignore'))])
preprocessor2 = ColumnTransformer(
    transformers=[
        ('minmax', minmax_transformer, year_ftrs),
        ('num', numeric_transformer2, num_ftrs),
        ('cat', categorical_transformer2, cat_ftrs)])

Preprocessor suitable for XGBoost, will produce missing values

Preprocessor2 is for random forest, LR, etc. It will fill missing values with 0

### Hyperparameter tuning w/ RandomizedSearchCV

In [13]:

def ML_BayesSearch_CV(X_og, y_og, groups_og, preprocessor, ML_algo, search_space, sample_size=None):
    # Loop for 5 random states
    # in each loop, split, preprocess, fit, and score
    random_states = [377, 575, 610, 777, 233]

    accuracy_scores = []
    precision_scores = []
    recall_scores = []
    f1_scores = []
    reports = []
    cms = []
    best_param = []
    best_score = []

    for i in range(1):
        this_rs = random_states[i]
        
        # Split into [train,val] and test sets (80%, 20%)
        sgss = StratifiedGroupKFold(n_splits=5, shuffle=True, random_state=this_rs)
        splits = enumerate(sgss.split(X_og, y_og, groups_og))

        print('Splitting dataset into train-val and test sets (80%, 20%)')
        i, (train_val_idx, test_idx) = next(splits)
        X_train_val, X_test = X_og.iloc[train_val_idx], X_og.iloc[test_idx]
        y_train_val, y_test = y_og.iloc[train_val_idx], y_og.iloc[test_idx]
        group_train_val, group_test = groups_og.iloc[train_val_idx], groups_og.iloc[test_idx]


        #full_length = len(y_train_val)
        # After getting test set, downsample train-val 
        if sample_size is None:
            pass
        elif sample_size <= 0.3*0.8:
            print('Sampling dataset for hyperparameter tuning, sample size of train set (80%): ', sample_size*100,'%')
            flat_idx = downsample_classes(y_train_val, group_train_val, random_state=this_rs)
            flat_X, flat_y, flat_groups = X_train_val.iloc[flat_idx], y_train_val.iloc[flat_idx], group_train_val.iloc[flat_idx]
            flat_prop = len(flat_idx)/len(y_train_val)
            if int(flat_prop/sample_size)<=1: # no need to do more sampling
                X, y = flat_X, flat_y
                groups = flat_groups
            else:
                sampler = StratifiedGroupKFold(n_splits=int(flat_prop/sample_size), shuffle=True, random_state=this_rs)
                splits = enumerate(sampler.split(flat_X, flat_y, flat_groups))
                i, (_, sampled_idx) = next(splits)
                X, y = flat_X.iloc[sampled_idx], flat_y.iloc[sampled_idx]
                groups = flat_groups.iloc[sampled_idx]
        else:
            pass
        X_train_val, y_train_val, group_train_val = X, y, groups
                

        # Split [train,val] into train and val sets (64%, 16%)
        sgss_cv = StratifiedGroupKFold(n_splits=4, shuffle=True, random_state=this_rs)
        pipe = make_pipeline(preprocessor, 
                             PCA(n_components=0.99, random_state=this_rs),
                             ML_algo)

        print('Random State Loop: ', i+1)
        print('Random State: ', this_rs)
        print('***Start Bayes Search***')

        bayes_search = BayesSearchCV(estimator=pipe,
                                    search_spaces=search_space,
                                    n_iter=25,  # Number of iterations
                                    cv=sgss_cv,       # Cross-validation strategy
                                    n_jobs=-1,  # Use all available cores
                                    scoring='f1_weighted',
                                    verbose=1,
                                    random_state=this_rs)


        bayes_search.fit(X_train_val, y_train_val, groups=group_train_val)        
    
        # Make predictions and calculate accuracy on the test set
        y_pred = bayes_search.predict(X_test)
        
        accuracy = accuracy_score(*vote_pred(y_test, y_pred, group_test))
        precision = precision_score(*vote_pred(y_test, y_pred, group_test), average="weighted")
        recall = recall_score(*vote_pred(y_test, y_pred, group_test), average="weighted")
        f1 = f1_score(*vote_pred(y_test, y_pred, group_test), average="weighted")

        report = classification_report(*vote_pred(y_test, y_pred, group_test))
        cm = confusion_matrix(*vote_pred(y_test, y_pred, group_test))

        accuracy_scores.append(accuracy)
        precision_scores.append(precision)
        recall_scores.append(recall)
        f1_scores.append(f1)

        reports.append(report)
        cms.append(cm)
        best_param.append(bayes_search.best_params_)
        best_score.append(bayes_search.best_score_)
        
    return accuracy_scores, precision_scores, recall_scores, f1_scores, cms, best_param, best_score

In [14]:
compustat[cat_ftrs] = compustat[cat_ftrs].astype(str)

groups = compustat['gvkey']
y = compustat['gsector_num']
X = compustat.drop(['gvkey','gsector','gsector_num','datafqtr'], axis=1)

X.shape

(2748178, 135)

#### XGBoost

#### Random Forest

In [15]:
search_space_rf = {
    'randomforestclassifier__n_estimators': Integer(100, 300),
    'randomforestclassifier__max_features': Categorical(['sqrt', 'log2']),
    'randomforestclassifier__max_depth': Integer(3, 35),
    'randomforestclassifier__min_samples_split': Integer(3, 50)
}
rf_clf = RandomForestClassifier()
acc_rf, pre_rf, rec_rf, f1_rf, \
    cm_rf, params_rf, bs_rf = ML_BayesSearch_CV(X, y, groups, preprocessor2,  
                                                rf_clf, search_space_rf, sample_size=0.04)
rf_results = {
    "accuracy": acc_rf,
    "precision": pre_rf,
    "recall": rec_rf,
    "f1_score": f1_rf,
    "conf_matrix": cm_rf,
    "params": params_rf,
    "best_score": bs_rf
}
with open('../results/rf_results.pkl', 'wb') as file:
    pickle.dump(rf_results, file)

Splitting dataset into train-val and test sets (80%, 20%)
Sampling dataset for hyperparameter tuning, sample size of train set (80%):  4.0 %
Random State Loop:  1
Random State:  377
***Start Bayes Search***
Fitting 4 folds for each of 1 candidates, totalling 4 fits
Fitting 4 folds for each of 1 candidates, totalling 4 fits
Fitting 4 folds for each of 1 candidates, totalling 4 fits
Fitting 4 folds for each of 1 candidates, totalling 4 fits
Fitting 4 folds for each of 1 candidates, totalling 4 fits
Fitting 4 folds for each of 1 candidates, totalling 4 fits
Fitting 4 folds for each of 1 candidates, totalling 4 fits
Fitting 4 folds for each of 1 candidates, totalling 4 fits
Fitting 4 folds for each of 1 candidates, totalling 4 fits
Fitting 4 folds for each of 1 candidates, totalling 4 fits
Fitting 4 folds for each of 1 candidates, totalling 4 fits
Fitting 4 folds for each of 1 candidates, totalling 4 fits
Fitting 4 folds for each of 1 candidates, totalling 4 fits
Fitting 4 folds for each o

In [17]:
rf_results_old = pickle.load(open('../results/rf_results.pkl', 'rb'))
rf_results_old

{'accuracy': [0.3866120218579235, 0.32432432432432434],
 'precision': [0.39653280324682, 0.4835263835263836],
 'recall': [0.3866120218579235, 0.32432432432432434],
 'f1_score': [0.38006405679977817, 0.2975975975975976],
 'conf_matrix': [array([[32,  4,  6,  2,  4,  5,  1,  1,  1,  4,  4],
         [12,  8,  8,  4,  6,  4,  0,  6,  1,  5,  1],
         [ 0,  1, 21,  2, 10,  5,  0, 10,  4,  3,  3],
         [ 1,  6, 10,  6, 11,  6,  0, 12,  5,  2, 10],
         [ 2,  5,  6,  5,  9,  4,  0,  6,  4,  4,  4],
         [ 4,  2,  3,  4,  9, 29,  0, 12,  4,  1,  2],
         [ 6,  1,  2,  1,  1,  7, 48,  3,  1,  0, 11],
         [ 7,  4, 14,  4,  3,  5,  1, 37,  6,  1,  3],
         [ 3,  1,  7,  2,  4,  7,  4, 16, 19,  6,  9],
         [ 4,  1,  8,  2,  5,  1,  0,  3,  1, 28,  5],
         [ 3,  1,  2,  1,  0,  4,  2,  3,  1,  1, 46]]),
  array([[0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
         [0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0],
         [0, 2, 0, 0, 0, 0, 0, 1, 0, 0, 0],
         [0, 0, 0, 1, 1, 0

In [18]:
np.mean([0.296,0.38])

0.33799999999999997

In [16]:
rf_results

{'accuracy': [0.29651310766098243],
 'precision': [0.34258457837687156],
 'recall': [0.29651310766098243],
 'f1_score': [0.2958998714402831],
 'conf_matrix': [array([[262,  47,   9,  34,  60,  66,  13,  32,  55,  73,  24],
         [157, 412,  69, 165, 238, 140,   8,  95,  77,  99,  38],
         [ 92, 186, 200, 278, 467, 136,  26, 300, 177, 180, 106],
         [ 67, 157,  85, 359, 327, 145,  31, 222, 211,  89,  94],
         [ 31,  60,  44, 126, 215,  73,   4,  84,  92,  45,  31],
         [ 60,  55,  41,  56, 117, 578,  18, 169, 116,  26,  18],
         [ 33,  17,   6,  24,  19,  38, 430,  18,  44,  19,  97],
         [ 69,  94, 106, 170, 245, 283,  11, 623, 194,  45,  37],
         [ 34,  16,  21,  51,  40,  73,   6,  87, 152,  42,  23],
         [ 22,   9,   6,  10,  19,  12,   7,  11,  12, 173,  16],
         [ 13,   6,   2,   7,   7,   5,   3,   4,   3,  15,  91]])],
 'params': [OrderedDict([('randomforestclassifier__max_depth', 29),
               ('randomforestclassifier__max_f

In [9]:
rf_results = {
    "accuracy": acc_rf,
    "precision": pre_rf,
    "recall": rec_rf,
    "f1_score": f1_rf,
    "conf_matrix": cm_rf,
    "params": params_rf,
    "best_score": bs_rf
}
with open('../results/rf_results.pkl', 'wb') as file:
    pickle.dump(rf_results, file)